In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

/var/folders/sf/bjt5p3d56qx14x468hvqxtvw0000gn/T/ipykernel_1331/4242059896.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(['EIN','NAME'],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 500].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Look at AFFILIATION value counts for binning
affiliate_count = application_df.AFFILIATION.value_counts()
affiliate_count

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [11]:
# Determine which values to replace if counts are less than ..?
replace_affiliate = list(affiliate_count[affiliate_count < 100].index)

# Replace in dataframe
for cls in replace_affiliate:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()


Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [12]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               3
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [13]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

/var/folders/sf/bjt5p3d56qx14x468hvqxtvw0000gn/T/ipykernel_1331/108820884.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

/var/folders/sf/bjt5p3d56qx14x468hvqxtvw0000gn/T/ipykernel_1331/2787429883.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 125
hidden_nodes_layer2 = 50

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               4750      
                                                                 
 dense_1 (Dense)             (None, 50)                6300      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75)

Epoch 1/75
804/804 [==============================] - 3s 2ms/step - loss: 0.5673 - accuracy: 0.7259
Epoch 2/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7305
Epoch 3/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5512 - accuracy: 0.7318
Epoch 4/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5494 - accuracy: 0.7337
Epoch 5/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7339
Epoch 6/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7346
Epoch 7/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7345
Epoch 8/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7353
Epoch 9/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7343
Epoch 10/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7354

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5602 - accuracy: 0.7252 - 567ms/epoch - 2ms/step
Loss: 0.5601575970649719, Accuracy: 0.7252478003501892


In [21]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization3/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization3/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75,callbacks=[cp_callback])

Epoch 1/75
 75/804 [=>............................] - ETA: 1s - loss: 0.5349 - accuracy: 0.7400
Epoch 1: saving model to checkpoints/Optimization3/weights.01.hdf5
184/804 [=====>........................] - ETA: 1s - loss: 0.5364 - accuracy: 0.7403
Epoch 1: saving model to checkpoints/Optimization3/weights.01.hdf5
299/804 [==========>...................] - ETA: 0s - loss: 0.5369 - accuracy: 0.7426
Epoch 1: saving model to checkpoints/Optimization3/weights.01.hdf5
377/804 [=============>................] - ETA: 0s - loss: 0.5329 - accuracy: 0.7459
Epoch 1: saving model to checkpoints/Optimization3/weights.01.hdf5
479/804 [================>.............] - ETA: 0s - loss: 0.5348 - accuracy: 0.7438
Epoch 1: saving model to checkpoints/Optimization3/weights.01.hdf5
590/804 [=====================>........] - ETA: 0s - loss: 0.5354 - accuracy: 0.7419
Epoch 1: saving model to checkpoints/Optimization3/weights.01.hdf5
679/804 [========================>.....] - ETA: 0s - loss: 0.5361 - accuracy:

267/804 [========>.....................] - ETA: 1s - loss: 0.5374 - accuracy: 0.7409
Epoch 7: saving model to checkpoints/Optimization3/weights.07.hdf5
371/804 [============>.................] - ETA: 0s - loss: 0.5381 - accuracy: 0.7390
Epoch 7: saving model to checkpoints/Optimization3/weights.07.hdf5
447/804 [===============>..............] - ETA: 0s - loss: 0.5371 - accuracy: 0.7393
Epoch 7: saving model to checkpoints/Optimization3/weights.07.hdf5
556/804 [===================>..........] - ETA: 0s - loss: 0.5368 - accuracy: 0.7395
Epoch 7: saving model to checkpoints/Optimization3/weights.07.hdf5
664/804 [=======================>......] - ETA: 0s - loss: 0.5362 - accuracy: 0.7408
Epoch 7: saving model to checkpoints/Optimization3/weights.07.hdf5
768/804 [===========================>..] - ETA: 0s - loss: 0.5358 - accuracy: 0.7414
Epoch 7: saving model to checkpoints/Optimization3/weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7413

450/804 [===============>..............] - ETA: 0s - loss: 0.5373 - accuracy: 0.7378
Epoch 13: saving model to checkpoints/Optimization3/weights.13.hdf5
536/804 [===================>..........] - ETA: 0s - loss: 0.5359 - accuracy: 0.7389
Epoch 13: saving model to checkpoints/Optimization3/weights.13.hdf5
634/804 [======================>.......] - ETA: 0s - loss: 0.5366 - accuracy: 0.7385
Epoch 13: saving model to checkpoints/Optimization3/weights.13.hdf5
743/804 [==========================>...] - ETA: 0s - loss: 0.5356 - accuracy: 0.7397
Epoch 13: saving model to checkpoints/Optimization3/weights.13.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5344 - accuracy: 0.7416
Epoch 14/75
 28/804 [>.............................] - ETA: 1s - loss: 0.5221 - accuracy: 0.7634
Epoch 14: saving model to checkpoints/Optimization3/weights.14.hdf5
127/804 [===>..........................] - ETA: 1s - loss: 0.5196 - accuracy: 0.7574
Epoch 14: saving model to checkpoints/Optimizatio

625/804 [======================>.......] - ETA: 0s - loss: 0.5357 - accuracy: 0.7398
Epoch 19: saving model to checkpoints/Optimization3/weights.19.hdf5
726/804 [==========================>...] - ETA: 0s - loss: 0.5332 - accuracy: 0.7423
Epoch 19: saving model to checkpoints/Optimization3/weights.19.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5338 - accuracy: 0.7417
Epoch 20/75
  1/804 [..............................] - ETA: 2s - loss: 0.5635 - accuracy: 0.6562
Epoch 20: saving model to checkpoints/Optimization3/weights.20.hdf5
114/804 [===>..........................] - ETA: 1s - loss: 0.5335 - accuracy: 0.7486
Epoch 20: saving model to checkpoints/Optimization3/weights.20.hdf5
223/804 [=======>......................] - ETA: 1s - loss: 0.5303 - accuracy: 0.7511
Epoch 20: saving model to checkpoints/Optimization3/weights.20.hdf5
308/804 [==========>...................] - ETA: 0s - loss: 0.5306 - accuracy: 0.7488
Epoch 20: saving model to checkpoints/Optimizatio

775/804 [===========================>..] - ETA: 0s - loss: 0.5337 - accuracy: 0.7420
Epoch 25: saving model to checkpoints/Optimization3/weights.25.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5339 - accuracy: 0.7416
Epoch 26/75
 88/804 [==>...........................] - ETA: 1s - loss: 0.5329 - accuracy: 0.7393
Epoch 26: saving model to checkpoints/Optimization3/weights.26.hdf5
174/804 [=====>........................] - ETA: 1s - loss: 0.5342 - accuracy: 0.7392
Epoch 26: saving model to checkpoints/Optimization3/weights.26.hdf5
297/804 [==========>...................] - ETA: 0s - loss: 0.5352 - accuracy: 0.7402
Epoch 26: saving model to checkpoints/Optimization3/weights.26.hdf5
387/804 [=============>................] - ETA: 0s - loss: 0.5336 - accuracy: 0.7418
Epoch 26: saving model to checkpoints/Optimization3/weights.26.hdf5
478/804 [================>.............] - ETA: 0s - loss: 0.5337 - accuracy: 0.7424
Epoch 26: saving model to checkpoints/Optimizatio

...

589/804 [====================>.........] - ETA: 0s - loss: 0.5368 - accuracy: 0.7391
Epoch 50: saving model to checkpoints/Optimization3/weights.50.hdf5
686/804 [========================>.....] - ETA: 0s - loss: 0.5353 - accuracy: 0.7411
Epoch 50: saving model to checkpoints/Optimization3/weights.50.hdf5
789/804 [============================>.] - ETA: 0s - loss: 0.5344 - accuracy: 0.7415
Epoch 50: saving model to checkpoints/Optimization3/weights.50.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5342 - accuracy: 0.7416
Epoch 51/75
 78/804 [=>............................] - ETA: 1s - loss: 0.5159 - accuracy: 0.7520
Epoch 51: saving model to checkpoints/Optimization3/weights.51.hdf5
178/804 [=====>........................] - ETA: 1s - loss: 0.5256 - accuracy: 0.7453
Epoch 51: saving model to checkpoints/Optimization3/weights.51.hdf5
284/804 [=========>....................] - ETA: 1s - loss: 0.5269 - accuracy: 0.7438
Epoch 51: saving model to checkpoints/Optimizatio

759/804 [===========================>..] - ETA: 0s - loss: 0.5322 - accuracy: 0.7418
Epoch 56: saving model to checkpoints/Optimization3/weights.56.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5330 - accuracy: 0.7413
Epoch 57/75
 59/804 [=>............................] - ETA: 1s - loss: 0.5265 - accuracy: 0.7452
Epoch 57: saving model to checkpoints/Optimization3/weights.57.hdf5
159/804 [====>.........................] - ETA: 1s - loss: 0.5313 - accuracy: 0.7390
Epoch 57: saving model to checkpoints/Optimization3/weights.57.hdf5
263/804 [========>.....................] - ETA: 1s - loss: 0.5308 - accuracy: 0.7398
Epoch 57: saving model to checkpoints/Optimization3/weights.57.hdf5
347/804 [===========>..................] - ETA: 0s - loss: 0.5320 - accuracy: 0.7405
Epoch 57: saving model to checkpoints/Optimization3/weights.57.hdf5
467/804 [================>.............] - ETA: 0s - loss: 0.5327 - accuracy: 0.7406
Epoch 57: saving model to checkpoints/Optimizatio

 31/804 [>.............................] - ETA: 1s - loss: 0.5217 - accuracy: 0.7550
Epoch 63: saving model to checkpoints/Optimization3/weights.63.hdf5
121/804 [===>..........................] - ETA: 1s - loss: 0.5303 - accuracy: 0.7505
Epoch 63: saving model to checkpoints/Optimization3/weights.63.hdf5
219/804 [=======>......................] - ETA: 0s - loss: 0.5324 - accuracy: 0.7430
Epoch 63: saving model to checkpoints/Optimization3/weights.63.hdf5
334/804 [===========>..................] - ETA: 0s - loss: 0.5345 - accuracy: 0.7422
Epoch 63: saving model to checkpoints/Optimization3/weights.63.hdf5
447/804 [===============>..............] - ETA: 0s - loss: 0.5329 - accuracy: 0.7435
Epoch 63: saving model to checkpoints/Optimization3/weights.63.hdf5
536/804 [===================>..........] - ETA: 0s - loss: 0.5327 - accuracy: 0.7445
Epoch 63: saving model to checkpoints/Optimization3/weights.63.hdf5
622/804 [======================>.......] - ETA: 0s - loss: 0.5334 - accuracy: 0.74

224/804 [=======>......................] - ETA: 1s - loss: 0.5279 - accuracy: 0.7458
Epoch 69: saving model to checkpoints/Optimization3/weights.69.hdf5
298/804 [==========>...................] - ETA: 0s - loss: 0.5297 - accuracy: 0.7427
Epoch 69: saving model to checkpoints/Optimization3/weights.69.hdf5
422/804 [==============>...............] - ETA: 0s - loss: 0.5332 - accuracy: 0.7402
Epoch 69: saving model to checkpoints/Optimization3/weights.69.hdf5
510/804 [==================>...........] - ETA: 0s - loss: 0.5349 - accuracy: 0.7396
Epoch 69: saving model to checkpoints/Optimization3/weights.69.hdf5
627/804 [======================>.......] - ETA: 0s - loss: 0.5343 - accuracy: 0.7403
Epoch 69: saving model to checkpoints/Optimization3/weights.69.hdf5
710/804 [=========================>....] - ETA: 0s - loss: 0.5341 - accuracy: 0.7412
Epoch 69: saving model to checkpoints/Optimization3/weights.69.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 

395/804 [=============>................] - ETA: 0s - loss: 0.5300 - accuracy: 0.7418
Epoch 75: saving model to checkpoints/Optimization3/weights.75.hdf5
480/804 [================>.............] - ETA: 0s - loss: 0.5318 - accuracy: 0.7404
Epoch 75: saving model to checkpoints/Optimization3/weights.75.hdf5
579/804 [====================>.........] - ETA: 0s - loss: 0.5325 - accuracy: 0.7405
Epoch 75: saving model to checkpoints/Optimization3/weights.75.hdf5
692/804 [========================>.....] - ETA: 0s - loss: 0.5319 - accuracy: 0.7417
Epoch 75: saving model to checkpoints/Optimization3/weights.75.hdf5
777/804 [===========================>..] - ETA: 0s - loss: 0.5330 - accuracy: 0.7416
Epoch 75: saving model to checkpoints/Optimization3/weights.75.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5327 - accuracy: 0.7417


In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5781 - accuracy: 0.7247 - 495ms/epoch - 2ms/step
Loss: 0.5781093239784241, Accuracy: 0.7246647477149963


In [23]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')